In [1]:
import json
import os
import shutil

##### map each serve to the closest hit based on the time difference between the serve's end and the hit's start. After mapping a serve to a hit, that hit is eliminated from consideration for subsequent serves.

#### if this does not work maybe use points and extract the serve and hit from points

In [2]:
def map_serve_to_hit(serves, hits, ace_threshold=400):
    serve_to_hit_mapping = {}

    for serve in serves:
        closest_hit = None
        closest_time_diff = float('inf')

        if serve["custom"]["Result"] == "In":
            # Check for hits following the serve
            for hit in hits:
                time_diff = abs(serve['end'] - hit['start'])
                
                if time_diff < closest_time_diff:
                    closest_time_diff = time_diff
                    closest_hit = hit

            # Determine if serve is an ace
            if closest_hit and closest_time_diff <= ace_threshold:
                serve_to_hit_mapping[serve['name']] = closest_hit['name']
                hits.remove(closest_hit)  # Eliminating the matched hit
            else:
                serve_to_hit_mapping[serve['name']] = "Ace or Bad retrieval"
        else:
            serve_to_hit_mapping[serve['name']] = serve["custom"]["Result"]

    return serve_to_hit_mapping

In [3]:
with open("Z:\Disertatie\data\V010.json", 'r') as file:
    json_content = file.read()

json_data = json.loads(json_content)

# Extracting the 'Serve' and 'Hit' data
serves = json_data['classes']['Serve']
hits = json_data['classes']['Hit']

# Mapping each serve to its closest hit
serve_hit_mapping = map_serve_to_hit(serves, hits, 100)
serve_hit_mapping

{'0001': 'Ace or Bad retrieval',
 '0002': 'Ace or Bad retrieval',
 '0003': '0001',
 '0004': 'Ace or Bad retrieval',
 '0005': '0002',
 '0006': '0003',
 '0007': '0014',
 '0008': 'Fault',
 '0009': 'Fault',
 '0010': '0019',
 '0011': '0023',
 '0012': 'Fault',
 '0013': '0028',
 '0014': '0032',
 '0015': '0033',
 '0016': '0036',
 '0017': 'Fault',
 '0018': '0038',
 '0019': '0039',
 '0020': '0040',
 '0021': '0043',
 '0022': '0048',
 '0023': '0051',
 '0024': 'Fault',
 '0025': '0056',
 '0026': '0058',
 '0027': 'Fault',
 '0028': '0060',
 '0029': '0074',
 '0030': 'Ace or Bad retrieval',
 '0031': 'Let',
 '0032': 'Ace or Bad retrieval',
 '0033': 'Fault',
 '0034': '0077',
 '0035': '0078',
 '0036': '0082',
 '0037': 'Let',
 '0038': '0085',
 '0039': '0086',
 '0040': '0094',
 '0041': 'Fault',
 '0042': '0100',
 '0043': 'Ace or Bad retrieval',
 '0044': '0106',
 '0045': 'Fault',
 '0046': '0110',
 '0047': 'Let',
 '0048': 'Ace or Bad retrieval',
 '0049': '0122',
 '0050': 'Ace or Bad retrieval',
 '0051': 'Fault'

In [6]:
def create_datapoints(serve_hit_mapping, data_path: str):
    datapoints_path = os.path.join(data_path, "Datapoints")

    # Create the Datapoints directory if it doesn't exist
    if not os.path.exists(datapoints_path):
        os.makedirs(datapoints_path)

    hit_ids = [item for item in serve_hit_mapping.values() if item.isdigit()]

    i = 1
    for serve_id, hit_id in serve_hit_mapping.items():
        datapoint_dir = os.path.join(datapoints_path, f"datapoint_{i}")
        os.makedirs(datapoint_dir, exist_ok=True)

        # Paths for Serve and Hit videos
        serve_video_path = os.path.join(data_path, "Serve", serve_id, "{}.mp4".format(serve_id))
        hit_video_path = os.path.join(data_path, "Hit", hit_id, "{}.mp4".format(hit_id))

        # Copy Serve and Hit videos to the new datapoint directory and rename them
        if os.path.exists(serve_video_path):
            shutil.copy(serve_video_path, os.path.join(datapoint_dir, "{}_s.mp4".format(serve_id)))
        
        if hit_id in hit_ids:
            if os.path.exists(hit_video_path):
                shutil.copy(hit_video_path, os.path.join(datapoint_dir, "{}_h.mp4".format(hit_id)))

        i+=1
    

### Based on how the datapoints look i might want to save the serve and the hit as a whole video directly from the point

In [7]:
data_path = "Z:/Disertatie/data/New_set/V010.mp4"
create_datapoints(serve_hit_mapping,data_path)